In [1]:
import json
import os
import pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/07 12:04:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
filepath = "./raw/"

files = []

for item in os.listdir(filepath):
    file = os.path.join(filepath,item)
    print(file)
    files.append(file)

files    
    # # open the specific file
    # with open(file) as f:
    #   data = json.load(f)
    #   print(json.dumps(data,indent=2))

./raw/20221205.json
./raw/20221206.json


['./raw/20221205.json', './raw/20221206.json']

In [4]:
from pyspark.sql.types import StructType,StructField,StringType,FloatType,DateType,ArrayType

In [5]:
# define a schema

schema_workout = StructType([
    StructField("date",DateType(),True),
    StructField("weight",FloatType(),True),
    StructField("weight_unit",StringType(),True),
    StructField("workout",StringType(),True)
])


In [6]:
# df = spark.read.schema(schema_workout)\
#                .json(path = files[0]
#                      ,multiLine=True)
      
df = spark.read.json(path = files[0],multiLine=True)
                        

In [7]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- weight: double (nullable = true)
 |-- weight_unit: string (nullable = true)
 |-- workout: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- details: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- reps: long (nullable = true)
 |    |    |    |    |-- unit: string (nullable = true)
 |    |    |    |    |-- weight: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- note: string (nullable = true)



In [8]:
df.show()

+----------+------+-----------+--------------------+
|      date|weight|weight_unit|             workout|
+----------+------+-----------+--------------------+
|2022-12-05|  85.4|         kg|[{[{8, lb, 0}, {8...|
+----------+------+-----------+--------------------+



In [9]:
from pyspark.sql import functions as F

You need to see [this](https://stackoverflow.com/questions/70087524/pyspark-how-to-parse-nested-json) for complex json:
- convert `struct` to `array` before unnesting

In [10]:
df2 = df.withColumn("workout",F.explode(F.col("workout"))).select("workout.*")
df2.show()

+--------------------+-----------+--------------------+
|             details|       name|                note|
+--------------------+-----------+--------------------+
|[{8, lb, 0}, {8, ...|    pull up|      slightly tired|
|[{10, lb, 95}, {1...|barbell row|    getting started!|
|[{10, lb, 45}, {1...|dumbell row|    getting started!|
|[{10, lb, 0}, {10...|       dips|dip targeting tricep|
+--------------------+-----------+--------------------+



In [11]:
df3 = df2.withColumn(colName = "details_2",col = F.explode(F.col("details")))\
         .withColumn(colName = "details_3",col = F.explode(F.col("details")))
df3.show()

+--------------------+-------+--------------+-----------+-----------+
|             details|   name|          note|  details_2|  details_3|
+--------------------+-------+--------------+-----------+-----------+
|[{8, lb, 0}, {8, ...|pull up|slightly tired| {8, lb, 0}| {8, lb, 0}|
|[{8, lb, 0}, {8, ...|pull up|slightly tired| {8, lb, 0}| {8, lb, 0}|
|[{8, lb, 0}, {8, ...|pull up|slightly tired| {8, lb, 0}| {5, lb, 0}|
|[{8, lb, 0}, {8, ...|pull up|slightly tired| {8, lb, 0}|{3, lb, 25}|
|[{8, lb, 0}, {8, ...|pull up|slightly tired| {8, lb, 0}|{3, lb, 25}|
|[{8, lb, 0}, {8, ...|pull up|slightly tired| {8, lb, 0}| {8, lb, 0}|
|[{8, lb, 0}, {8, ...|pull up|slightly tired| {8, lb, 0}| {8, lb, 0}|
|[{8, lb, 0}, {8, ...|pull up|slightly tired| {8, lb, 0}| {5, lb, 0}|
|[{8, lb, 0}, {8, ...|pull up|slightly tired| {8, lb, 0}|{3, lb, 25}|
|[{8, lb, 0}, {8, ...|pull up|slightly tired| {8, lb, 0}|{3, lb, 25}|
|[{8, lb, 0}, {8, ...|pull up|slightly tired| {5, lb, 0}| {8, lb, 0}|
|[{8, lb, 0}, {8, ..

In [12]:
df3 = df2.withColumn(colName = "details",col = F.explode(F.col("details")))
df3.show()

+------------+-----------+--------------------+
|     details|       name|                note|
+------------+-----------+--------------------+
|  {8, lb, 0}|    pull up|      slightly tired|
|  {8, lb, 0}|    pull up|      slightly tired|
|  {5, lb, 0}|    pull up|      slightly tired|
| {3, lb, 25}|    pull up|      slightly tired|
| {3, lb, 25}|    pull up|      slightly tired|
|{10, lb, 95}|barbell row|    getting started!|
|{10, lb, 95}|barbell row|    getting started!|
|{10, lb, 95}|barbell row|    getting started!|
| {8, lb, 95}|barbell row|    getting started!|
|{10, lb, 45}|dumbell row|    getting started!|
|{10, lb, 45}|dumbell row|    getting started!|
|{10, lb, 45}|dumbell row|    getting started!|
| {8, lb, 45}|dumbell row|    getting started!|
| {10, lb, 0}|       dips|dip targeting tricep|
| {10, lb, 0}|       dips|dip targeting tricep|
+------------+-----------+--------------------+



In [13]:
F.explode(F.col("details"))

Column<'explode(details)'>

In [14]:
df3 = df2.withColumn(colName = "reps",col = F.explode(F.col("details")))
df3.show()

+--------------------+-----------+--------------------+------------+
|             details|       name|                note|        reps|
+--------------------+-----------+--------------------+------------+
|[{8, lb, 0}, {8, ...|    pull up|      slightly tired|  {8, lb, 0}|
|[{8, lb, 0}, {8, ...|    pull up|      slightly tired|  {8, lb, 0}|
|[{8, lb, 0}, {8, ...|    pull up|      slightly tired|  {5, lb, 0}|
|[{8, lb, 0}, {8, ...|    pull up|      slightly tired| {3, lb, 25}|
|[{8, lb, 0}, {8, ...|    pull up|      slightly tired| {3, lb, 25}|
|[{10, lb, 95}, {1...|barbell row|    getting started!|{10, lb, 95}|
|[{10, lb, 95}, {1...|barbell row|    getting started!|{10, lb, 95}|
|[{10, lb, 95}, {1...|barbell row|    getting started!|{10, lb, 95}|
|[{10, lb, 95}, {1...|barbell row|    getting started!| {8, lb, 95}|
|[{10, lb, 45}, {1...|dumbell row|    getting started!|{10, lb, 45}|
|[{10, lb, 45}, {1...|dumbell row|    getting started!|{10, lb, 45}|
|[{10, lb, 45}, {1...|dumbell row|

In [15]:
df4 = df3.select('details.*')
df4.show()

AnalysisException: Can only star expand struct data types. Attribute: `ArrayBuffer(details)`